<a href="https://colab.research.google.com/github/mdiogof/Contrata-Bot/blob/main/Contrata_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
%pip -q install google-genai

In [44]:
%pip -q install google-adk

In [45]:
import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [70]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types
from datetime import date
import textwrap
from IPython.display import Markdown, display
from IPython.display import display, HTML
import requests
import warnings
import json
import re

warnings.filterwarnings("ignore")

In [48]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [49]:
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate = lambda _: True))

In [65]:
def agente_buscador(curso, local, cargo, data_de_hoje):
  buscador = Agent(
      name = "agente_buscador",
      model = "gemini-2.0-flash",
      instruction = """
      Você é um assistente de busca. Sua função é usar a ferramenta de pesquisa
      do Google (google_search) para encontrar vagas de emprego, baseando-se no
      curso informado abaixo. Considere também o local informado e o cargo
      desejado na empresa. Essa busca deve focar em ofertas de emprego atuais,
      de no máximo duas semanas antes da data de hoje. Confira se a vaga ainda
      está aberta e inclua a data de publicação de cada vaga.
      """,
      description = "Agente que encontra vagas de emprego no Google",
      tools = [google_search]
  )

  entrada_agente_buscador = f"Curso: {curso}\nLocal: {local}\nCargo: {cargo}\nData de hoje: {data_de_hoje}"
  vagas = call_agent(buscador, entrada_agente_buscador)
  return vagas

In [68]:
def agente_seletor(vagas_encontradas, periodo):
    seletor = Agent(
        name="agente_seletor",
        model="gemini-2.0-flash",
        instruction="""
        Você é um agente seletor de vagas de emprego para estudantes do ensino superior.
        Sua função é refinar a seleção das vagas com base na relevância em relação ao período de curso informado.

        🔸 Responda apenas com um JSON puro como este:
        [
          {"titulo": "Estágio em Desenvolvimento - Empresa X", "link": "https://example.com/vaga1"},
          {"titulo": "Estágio em Hardware - Empresa Y", "link": "https://example.com/vaga2"}
        ]

        ⚠️ Não inclua texto explicativo, markdown ou código markdown (```json).
        ⚠️ Não invente vagas, use apenas as fornecidas.
        """,
        description="Agente que refina a busca de vagas de emprego",
        tools=[google_search]
    )

    entrada = f"Vagas: {vagas_encontradas}\nPeríodo: {periodo}"
    resposta = call_agent(seletor, entrada)

    # Extração de JSON usando expressão regular
    match = re.search(r'\[.*?\]', resposta, re.DOTALL)
    if match:
        try:
            vagas_json = json.loads(match.group())
            return vagas_json
        except json.JSONDecodeError:
            print("⚠️ Erro ao decodificar JSON.")
            return []
    else:
        print("⚠️ Nenhum JSON válido encontrado na resposta.")
        return []

In [63]:
def exibidor_de_links(vagas_selecionadas):
    # Gerar os botões HTML com base nas vagas
    botoes_html = ""
    for vaga in vagas_selecionadas:
        titulo = vaga["titulo"]
        link = vaga["link"]
        botoes_html += f"""
        <a href="{link}" target="_blank" style="text-decoration:none;">
            <button style="
                background-color:#4CAF50;
                color:white;
                padding:10px 20px;
                margin:5px;
                border:none;
                border-radius:5px;
                cursor:pointer;
                font-size:16px;
            ">{titulo}</button>
        </a>
        """

    # Exibir os botões
    return botoes_html

In [74]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("""Olá, você acessou o Contrata Bot!\n
Antes de te mostrar a oportunidade certa, vou precisar de algumas informações.""")
curso = input("Primeiro, digite o curso que você faz: ")
periodo = input("E em qual período de curso você está? ")
local = input("Agora, digite a região que você quer encontrar vagas: ")
cargo = input("Por fim, digite o cargo que você quer: ")

if not curso or not local or not cargo:
  print("Você precisa preencher todos os campos!")
else:
  print("Perfeito, vou procurar vagas para você!")

  vagas = agente_buscador(curso, local, cargo, data_de_hoje)
  print("\n--- Vagas encontradas pelo agente 1 (Buscador) ---\n")
  display(to_markdown(vagas))
  print("----------------------------------------------------\n")

  vagas_selecionadas = agente_seletor(vagas, periodo)
  print("\n--- Essas são as vagas mais relevantes para você no momento---\n")
  display(HTML(exibidor_de_links(vagas_selecionadas)))

Olá, você acessou o Contrata Bot!

Antes de te mostrar a oportunidade certa, vou precisar de algumas informações.
Primeiro, digite o curso que você faz: Relações Internacionais
E em qual período de curso você está? Primeiro Período
Agora, digite a região que você quer encontrar vagas: Brasília
Por fim, digite o cargo que você quer: Estágio
Perfeito, vou procurar vagas para você!

--- Vagas encontradas pelo agente 1 (Buscador) ---



> Encontrei algumas oportunidades de estágio em Relações Internacionais em Brasília, publicadas recentemente:
> 
> *   **Banco Central do Brasil:** O Super Estágios está divulgando uma vaga de estágio para estudantes de Relações Internacionais a partir do 1º período. A bolsa é de R$ 1125,69, mais auxílio transporte de R$ 10,00 por dia. O estágio é de 6 horas, nos turnos matutino ou vespertino, no bairro Asa Sul. As atividades incluem atendimento telefônico, organização de documentos, elaboração de planilhas e relatórios, organização de eventos internacionais, comunicação em idiomas estrangeiros e relacionamento com instituições internacionais.
> *   **CNA (Confederação da Agricultura e Pecuária do Brasil):** A CNA está contratando estagiário para atuação na Diretoria de Relações Internacionais, com foco na defesa de interesses do agro brasileiro em mercados internacionais. Período: 14/03/2025 – 28/03/2025.
> *   **Grupo de Cooperação Internacional de Universidades Brasileiras (GCUB):** O GCUB está com uma vaga de estágio. Inscrições até 24 de janeiro de 2025.
> *   **Oportunidades DF:** Há uma vaga de estágio em Relações Internacionais, com bolsa auxílio de R$ 1.500,00, carga horária de 6 horas diárias, em formato presencial no Setor Bancário Norte. É necessário estar cursando a partir do 5º semestre e ter fluência em inglês e espanhol.
> 
> Além dessas, o Instituto de Relações Internacionais da Universidade de Brasília (IREL) promove eventos e atividades. A UFMG (Universidade Federal de Minas Gerais) apresentou possibilidades de cooperação nas áreas de ciências sociais e humanas em Brasília.
> 


----------------------------------------------------


--- Essas são as vagas mais relevantes para você no momento---

